# Data prepration in python

# Data reading

In [3]:
import glob
import gensim
import numpy as np

In [5]:
ff=glob.glob("Raw_data/*.txt")

In [6]:
len(ff)

1000

In [7]:
ff[0]

'Raw_data\\Text0001.txt'

In [8]:
raw_doc=[]
for file in ff:
    try:
        f=open(file,"r")
        raw_doc.append(f.read())
    except:
        pass

In [9]:
len(raw_doc)

764

In [10]:
clean_texts=[]
for text in raw_doc:
    #This lowercases, tokenizes, 
    clean_texts.append(gensim.utils.simple_preprocess(text))

In [11]:
clean_texts[0][:10]

['finalists',
 'in',
 'the',
 'apertura',
 'play',
 'offs',
 'toluca',
 'had',
 'drawn',
 'their']

In [12]:
Dictionary=gensim.corpora.Dictionary(clean_texts)
print(len(Dictionary))

20985


In [13]:
for i in range(10):
    print(i,Dictionary[i])

0 against
1 ahead
2 aldo
3 all
4 also
5 apertura
6 arizala
7 benitez
8 britos
9 but


In [14]:
Dictionary.token2id['apertura']

5

In [15]:
import numpy as np
np.array(clean_texts[0]).T

array(['finalists', 'in', 'the', 'apertura', 'play', 'offs', 'toluca',
       'had', 'drawn', 'their', 'first', 'two', 'clausura', 'games',
       'but', 'got', 'off', 'to', 'good', 'start', 'when', 'edgar',
       'benitez', 'put', 'them', 'ahead', 'in', 'the', 'th', 'minute',
       'matias', 'britos', 'levelled', 'minutes', 'later', 'but', 'lucas',
       'silva', 'netted', 'minutes', 'from', 'the', 'end', 'to', 'ensure',
       'the', 'visitors', 'took', 'all', 'three', 'points', 'franco',
       'arizala', 'scored', 'minutes', 'from', 'time', 'to', 'ensure',
       'jaguares', 'claimed', 'their', 'first', 'point', 'with', 'draw',
       'against', 'monterrey', 'who', 'had', 'opened', 'the', 'scoring',
       'through', 'aldo', 'de', 'nigris', 'hosts', 'jaguares', 'also',
       'had', 'jorge', 'rodriguez', 'sent', 'off', 'in', 'the', 'closing',
       'moments'], dtype='<U9')

Now we will create a corpus. A corpus is a list of bags of words. A bag-of-words representation for a document just lists the number of times each word occurs in the document.

In [16]:
corpus=[Dictionary.doc2bow(text) for text in clean_texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 3), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 6), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)]


In [17]:
tf_idf=gensim.models.TfidfModel(corpus)
print(tf_idf)

TfidfModel(num_docs=764, num_nnz=206535)


In [18]:
ll=np.argsort(np.array([w[1] for w in tf_idf[corpus[0]]]))

In [19]:
np.array([Dictionary[w] for w in ll]).T

array(['the', 'in', 'to', 'with', 'who', 'but', 'two', 'when', 'all',
       'time', 'from', 'also', 'three', 'play', 'good', 'against',
       'first', 'games', 'them', 'their', 'got', 'points', 'had', 'end',
       'point', 'through', 'put', 'start', 'took', 'th', 'scored', 'off',
       'later', 'ahead', 'minute', 'scoring', 'sent', 'draw', 'de',
       'moments', 'opened', 'claimed', 'benitez', 'visitors', 'drawn',
       'closing', 'hosts', 'silva', 'minutes', 'lucas', 'rodriguez',
       'jorge', 'netted', 'levelled', 'edgar', 'offs', 'monterrey',
       'aldo', 'finalists', 'nigris', 'toluca', 'franco', 'matias',
       'clausura', 'arizala', 'britos', 'apertura', 'ensure', 'jaguares'],
      dtype='<U9')

In [21]:
similarity_object=gensim.similarities.Similarity('Raw_data/',tf_idf[corpus],num_features=len(Dictionary))

In [22]:
similarity_object.num_best = 3
similarity_object[tf_idf[corpus[0]]]

[(0, 0.9999998807907104),
 (626, 0.27305036783218384),
 (12, 0.14963755011558533)]

Now create a query document and convert it to tf-idf.

In [23]:
text=raw_doc[8]
query_doc=gensim.utils.simple_preprocess(text)
query_doc_bow=Dictionary.doc2bow(query_doc)
query_doc_tf_idf=tf_idf[query_doc_bow]

In [24]:
similarity_scores=list(similarity_object[query_doc_tf_idf])

In [25]:
max_score=max(similarity_scores)

In [26]:
print(max_score)

(536, 0.1600104123353958)


In [27]:
similarity_scores.index(max_score)

2

In [28]:
sorted_score=sorted(similarity_scores,reverse=True)
print(sorted_score[0])
print(sorted_score[1])

(536, 0.1600104123353958)
(37, 0.19432590901851654)


In [29]:
similarity_scores.index(sorted_score[1])

1

In [30]:
raw_doc[40]

'Eurosport - Megadeal mit Nike: Rory McIlroy\nRelated Content\nEurosport - Tue, Feb 19, 2013 17:42 GMT\nRelated Content\nBet on Golf\nThe Northern Irishman has not competed since his premature exit from the European Tour\'s Abu Dhabi Championship but said he was feeling refreshed after mixing some welcome holiday time with 10 days of practice sessions over the past month.\n"I\'m actually much happier with how I\'m swinging the club," world number one McIlroy told reporters at Dove Mountain on Tuesday while preparing for Wednesday\'s opening round.\n"The clubs were performing fine in Abu Dhabi, it was just the fact that I wasn\'t swinging at my best. I feel like I\'ve turned a corner with my swing. I\'ve got it back on track, and that\'s ultimately what\'s going to help me play better."\nWidely panned for switching his golf brand last month from Titleist to Nike in a lucrative deal reported to be worth as much as $250 million over 10 years, McIlroy said he now tried to blinker himself f

# Text Summarization

In [31]:
from gensim.summarization import summarize
from gensim.summarization import keywords

In [32]:
print(summarize(raw_doc[40],word_count=100))

Widely panned for switching his golf brand last month from Titleist to Nike in a lucrative deal reported to be worth as much as $250 million over 10 years, McIlroy said he now tried to blinker himself from criticism as much as possible.
McIlroy, who has put a new putter and a slightly heavier driver in his bag, said he had prepared himself for likely criticism when his switch to Nike was announced with much fanfare before last month's Abu Dhabi Championship.
"But I've had a nice few weeks off and I feel like I'm coming into this event prepared, ready and swinging the club well.
